# Experiment : Network growth

## Init the Ray client

In [1]:
# Imports
from clases.red_rddas_model import RedRddasModel
import ray
import time
import pandas as pd

# Ray Configurations
# runtime_env = {"working_dir": "/home/carlos/Documents/RESEARCH/SynEstRDDA", "pip": ["requests", "pendulum==2.1.2"]}
# ray.init(address='ray://172.17.163.244:10001', runtime_env=runtime_env , log_to_driver=False)
ray.init(log_to_driver=False, num_cpus=12)

RayContext(dashboard_url='', python_version='3.8.10', ray_version='1.13.0', ray_commit='e4ce38d001dbbe09cd21c497fedd03d692b2be3e', address_info={'node_ip_address': '172.17.163.244', 'raylet_ip_address': '172.17.163.244', 'redis_address': None, 'object_store_address': '/tmp/ray/session_2022-07-05_22-07-00_707653_245157/sockets/plasma_store', 'raylet_socket_name': '/tmp/ray/session_2022-07-05_22-07-00_707653_245157/sockets/raylet', 'webui_url': '', 'session_dir': '/tmp/ray/session_2022-07-05_22-07-00_707653_245157', 'metrics_export_port': 57998, 'gcs_address': '172.17.163.244:65531', 'address': '172.17.163.244:65531', 'node_id': '06b32ea84e7bfc00e105f3ab72362ef652fe9a433975f7ca5357c0af'})

In [2]:
# Take the time of all Experiment
v_begin_exp = time.time()
# Experiment for RDD variation
n_experiments = 500
l_experiments = []
for cont_experiment in range(1,n_experiments+1):
    print("============================")
    print("Experiment:", cont_experiment)
    print("============================")

    # Variable Parameters
    n_rddas_min = 3
    n_rddas_max = 10

    # Fixed Parameters
    n_of_variables_rdda = 5
    n_of_signals_rdda = 2
    n_exit_variables = 2
    n_clauses_function = 2
    type_network = "ALEATORY"

    # List of Result for the Experiments
    l_res_sample = []

    v_n_network = 1
    for n_of_rdds in range(n_rddas_min, n_rddas_max + 1):
        print("Number of Network:", v_n_network)
        print("-------------------------------")

        # generate the RDDAs of the Network of RDDAs
        print("generating the Network of RDDAs ...")
        oRedRddasModel = RedRddasModel(n_of_rdds, n_of_variables_rdda, n_of_signals_rdda, n_exit_variables,
                                       n_clauses_function)

        # Generate the RDDs
        print("generating the rdds ...")
        oRedRddasModel.generate_rddas(type_network=type_network)

        # Calculate the Attractors by RDDA and by Signal
        v_begin_0 = time.time()
        result = RedRddasModel.find_attractors_rddas_ray.remote(oRedRddasModel)
        oRedRddasModel = ray.get(result)
        v_end_0 = time.time()
        v_time_0 = v_end_0 - v_begin_0

        # Calculate the Attractors by RDDA and by Signal
        v_begin_1 = time.time()
        result = RedRddasModel.calculation_compatible_pairs.remote(oRedRddasModel)
        oRedRddasModel = ray.get(result)
        v_end_1 = time.time()
        v_time_1 = v_end_1 - v_begin_1

        # # Calculate the Attractors by RDDA and by Signal with iterative Method
        # v_begin_1 = time.time()
        # # result = RedRddasModel.assembly_attractor_fields_iterative.remote(oRedRddasModel)
        # # oRedRddasModel = ray.get(result)
        # v_end_1 = time.time()
        # v_time_1 = v_end_1 - v_begin_1

        # Calculate the Attractors by RDDA and by Signal with optimized Method
        v_begin_2 = time.time()
        result = RedRddasModel.assembly_attractor_fields_optimized.remote(oRedRddasModel)
        oRedRddasModel = ray.get(result)
        v_end_2 = time.time()
        v_time_2 = v_end_2 - v_begin_2

        # Save the results for the experiment , numeric and time indicators
        res_dict = {
                    "n_network": v_n_network,
                    "n_rdds": n_of_rdds,
                    "n_rdda_attractors": len(oRedRddasModel.d_global_rdda_attractor.items()),
                    "t_find_attractors_method": v_time_0,
                    "n_pair_attractors": len(oRedRddasModel.list_attractors_pairs),
                    "t_comp_paris_method": v_time_1,
                    "n_attractor_fields": len(oRedRddasModel.attractor_fields),
                    "t_optimized_method": v_time_2
                    }
        l_res_sample.append(res_dict)
        v_n_network = v_n_network + 1

    # Add the sample data to pandas dataframe
    df = pd.DataFrame.from_dict(l_res_sample)
    l_experiments.append(df)
print("END EXPERIMENT")
# Take the time of the experiment
v_end_exp = time.time()
v_time_exp = v_end_exp - v_begin_exp
print("Time experiment (in seconds): ", v_time_exp )

Experiment: 1
Number of Network: 1
-------------------------------
generating the Network of RDDAs ...
generating the rdds ...
Number of Network: 2
-------------------------------
generating the Network of RDDAs ...
generating the rdds ...
Number of Network: 3
-------------------------------
generating the Network of RDDAs ...
generating the rdds ...
Number of Network: 4
-------------------------------
generating the Network of RDDAs ...
generating the rdds ...
Number of Network: 5
-------------------------------
generating the Network of RDDAs ...
generating the rdds ...
Number of Network: 6
-------------------------------
generating the Network of RDDAs ...
generating the rdds ...
Number of Network: 7
-------------------------------
generating the Network of RDDAs ...
generating the rdds ...
Number of Network: 8
-------------------------------
generating the Network of RDDAs ...
generating the rdds ...
Experiment: 2
Number of Network: 1
-------------------------------
generating the 

## Save the experiment data in csv, using pandas Dataframe

In [3]:
pf_res = pd.concat(l_experiments, keys=range(1,n_experiments+1), names=["n_sample","n_aux"], ignore_index=False)
pf_res.reset_index(drop=True, inplace=True, level=1)
pf_res

,n_network,n_rdds,n_rdda_attractors,t_find_attractors_method,n_pair_attractors,t_comp_paris_method,n_attractor_fields,t_optimized_method
n_sample,,,,,,,,
1,1,3,14,1.127742,27,0.036490,5,0.016521
1,2,4,29,0.567921,160,0.044149,24,0.037182
1,3,5,49,0.785318,275,0.062197,0,0.004468
1,4,6,50,0.936676,380,0.095642,27,0.244904
1,5,7,54,0.887210,231,0.058252,0,0.003945
...,...,...,...,...,...,...,...,...
500,4,6,40,0.978942,187,0.056139,10,0.119527
500,5,7,35,0.879762,101,0.035724,15,0.041956
500,6,8,52,1.192511,175,0.052122,0,0.004834


In [4]:
path = "exp1_network_growth_data.csv"
pf_res.to_csv(path)
print("Experiment saved in:", path)

Experiment saved in: exp1_network_growth_data.csv
